In [9]:
import json
#import _cvxcore
#import cvxpy
import fancyimpute
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler,SimpleFill
import numpy as np
import pandas as pd
import random
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pickle
from sklearn.model_selection import train_test_split
from datetime import timedelta
from datetime import datetime

In [10]:
mimic=pd.read_csv('../initial/labs_cohort2.csv',header=0)

In [12]:
mimic.head()

,subject_id,charttime,ALBUMIN,ANION GAP,BANDS,BICARBONATE,BILIRUBIN,BUN,CHLORIDE,CREATININE,...,HEMOGLOBIN,INR,LACTATE,PaCO2,PLATELET,POTASSIUM,PT,PTT,SODIUM,WBC
0,4,2191-03-15T14:12:00Z,NaN,17.0,NaN,24.0,2.2,9.0,97.0,0.5,...,11.5,NaN,NaN,NaN,207.0,3.1,NaN,NaN,135.0,9.7
1,4,2191-03-15T15:50:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,12.3,31.3,NaN,NaN
2,4,2191-03-15T21:23:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.5,NaN,2.1,28.0,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2191-03-16T05:42:00Z,2.8,15.0,NaN,21.0,1.9,10.0,108.0,0.5,...,10.6,1.1,NaN,NaN,201.0,3.3,12.8,33.2,141.0,9.7
4,4,2191-03-17T06:00:00Z,NaN,15.0,NaN,21.0,NaN,16.0,106.0,0.4,...,10.5,NaN,NaN,NaN,258.0,3.8,NaN,NaN,138.0,7.3


In [13]:
mimic.shape

(206876, 22)

In [16]:
(mimic.isnull().sum()).sum()

2815989

In [17]:
206876*20

4137520

In [8]:
2815989/4137520

0.6805982810959222

In [18]:
len(mimic['subject_id'].unique())

17111

In [19]:
mimic.groupby(['subject_id']).count()

,charttime,ALBUMIN,ANION GAP,BANDS,BICARBONATE,BILIRUBIN,BUN,CHLORIDE,CREATININE,GLUCOSE,...,HEMOGLOBIN,INR,LACTATE,PaCO2,PLATELET,POTASSIUM,PT,PTT,SODIUM,WBC
subject_id,,,,,,,,,,,,,,,,,,,,,
4,5,1,3,0,3,2,3,3,3,3,...,4,2,1,1,3,3,2,2,3,3
9,14,0,4,0,4,2,4,5,4,5,...,6,3,5,6,5,10,3,3,5,5
12,27,0,3,0,3,0,3,9,3,11,...,13,9,11,15,6,10,9,9,9,4
19,6,0,3,0,3,1,3,4,3,4,...,4,2,2,1,3,4,2,2,4,3
21,20,0,6,0,6,1,6,6,6,8,...,8,6,5,7,6,8,6,6,6,5
23,12,0,2,0,2,0,3,2,3,7,...,2,0,0,6,2,8,0,0,2,2
25,19,1,8,0,8,1,8,8,8,8,...,7,4,4,8,6,10,4,4,8,6
26,4,0,4,0,4,1,4,4,4,4,...,3,2,0,0,3,4,2,2,4,3
31,7,1,4,0,4,1,4,4,4,5,...,3,4,2,3,4,4,4,4,4,3


In [20]:
mimic[mimic['subject_id']==4]

,subject_id,charttime,ALBUMIN,ANION GAP,BANDS,BICARBONATE,BILIRUBIN,BUN,CHLORIDE,CREATININE,...,HEMOGLOBIN,INR,LACTATE,PaCO2,PLATELET,POTASSIUM,PT,PTT,SODIUM,WBC
0,4,2191-03-15T14:12:00Z,NaN,17.0,NaN,24.0,2.2,9.0,97.0,0.5,...,11.5,NaN,NaN,NaN,207.0,3.1,NaN,NaN,135.0,9.7
1,4,2191-03-15T15:50:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1.0,NaN,NaN,NaN,NaN,12.3,31.3,NaN,NaN
2,4,2191-03-15T21:23:00Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.5,NaN,2.1,28.0,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2191-03-16T05:42:00Z,2.8,15.0,NaN,21.0,1.9,10.0,108.0,0.5,...,10.6,1.1,NaN,NaN,201.0,3.3,12.8,33.2,141.0,9.7
4,4,2191-03-17T06:00:00Z,NaN,15.0,NaN,21.0,NaN,16.0,106.0,0.4,...,10.5,NaN,NaN,NaN,258.0,3.8,NaN,NaN,138.0,7.3


In [22]:
mimic.isnull().sum()

subject_id          0
charttime           0
ALBUMIN        192653
ANION GAP      140470
BANDS          201799
BICARBONATE    138675
BILIRUBIN      185184
BUN            137018
CHLORIDE       123195
CREATININE     136736
GLUCOSE        101319
HEMATOCRIT     103311
HEMOGLOBIN     119184
INR            155831
LACTATE        157801
PaCO2          120719
PLATELET       137673
POTASSIUM       95751
PT             155831
PTT            153685
SODIUM         118747
WBC            140407
dtype: int64

In [23]:
mimic = mimic.sort_values(by=['subject_id','charttime'])

In [25]:
mimic_pad = pd.concat([
        d.reset_index(drop=True).reindex(range(20)).assign(subject_id=n)
        for n, d in mimic.groupby(['subject_id'])
        ], ignore_index=True)

In [26]:
mimic_pad = mimic_pad.fillna(0)

In [28]:
rows=mimic_pad.groupby('subject_id').count()['charttime']-20

In [29]:
rows=pd.DataFrame(rows)

In [30]:
rows=rows.reset_index()

In [31]:
mimic_pad.head()

,subject_id,charttime,ALBUMIN,ANION GAP,BANDS,BICARBONATE,BILIRUBIN,BUN,CHLORIDE,CREATININE,...,HEMOGLOBIN,INR,LACTATE,PaCO2,PLATELET,POTASSIUM,PT,PTT,SODIUM,WBC
0,4,2191-03-15T14:12:00Z,0.0,17.0,0.0,24.0,2.2,9.0,97.0,0.5,...,11.5,0.0,0.0,0.0,207.0,3.1,0.0,0.0,135.0,9.7
1,4,2191-03-15T15:50:00Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,12.3,31.3,0.0,0.0
2,4,2191-03-15T21:23:00Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.5,0.0,2.1,28.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,2191-03-16T05:42:00Z,2.8,15.0,0.0,21.0,1.9,10.0,108.0,0.5,...,10.6,1.1,0.0,0.0,201.0,3.3,12.8,33.2,141.0,9.7
4,4,2191-03-17T06:00:00Z,0.0,15.0,0.0,21.0,0.0,16.0,106.0,0.4,...,10.5,0.0,0.0,0.0,258.0,3.8,0.0,0.0,138.0,7.3


In [32]:
mimic_pad.shape

(342220, 22)

In [33]:
mimic_pad[mimic_pad['subject_id']==4]

,subject_id,charttime,ALBUMIN,ANION GAP,BANDS,BICARBONATE,BILIRUBIN,BUN,CHLORIDE,CREATININE,...,HEMOGLOBIN,INR,LACTATE,PaCO2,PLATELET,POTASSIUM,PT,PTT,SODIUM,WBC
0,4,2191-03-15T14:12:00Z,0.0,17.0,0.0,24.0,2.2,9.0,97.0,0.5,...,11.5,0.0,0.0,0.0,207.0,3.1,0.0,0.0,135.0,9.7
1,4,2191-03-15T15:50:00Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,12.3,31.3,0.0,0.0
2,4,2191-03-15T21:23:00Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.5,0.0,2.1,28.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,2191-03-16T05:42:00Z,2.8,15.0,0.0,21.0,1.9,10.0,108.0,0.5,...,10.6,1.1,0.0,0.0,201.0,3.3,12.8,33.2,141.0,9.7
4,4,2191-03-17T06:00:00Z,0.0,15.0,0.0,21.0,0.0,16.0,106.0,0.4,...,10.5,0.0,0.0,0.0,258.0,3.8,0.0,0.0,138.0,7.3
5,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [34]:
final=pd.DataFrame()
for seq in range(rows.shape[0]):
    #print(rows.iloc[seq,0])
    #print(rows.iloc[seq,1])
    
    temp=mimic_pad[mimic_pad['subject_id']==rows.iloc[seq,0]]
    #print(temp.shape)
    nrows = temp.shape[0]-rows.iloc[seq,1]
    temp=temp.iloc[0:nrows,:]
    #print(temp.shape)
    #print(temp.shape[0]%20)
    final=pd.concat([final,temp])

In [23]:
final.shape

(342220, 22)

In [24]:
final.shape

(342220, 22)

In [25]:
temp=final.iloc[:,2:].shape

In [26]:
temp=final.iloc[:,2:]

In [27]:
temp['WBC'].shape

(342220,)

In [28]:
(temp[temp['HEMOGLOBIN']==0].count())

ALBUMIN        259189
ANION GAP      259189
BANDS          259189
BICARBONATE    259189
BILIRUBIN      259189
BUN            259189
CHLORIDE       259189
CREATININE     259189
GLUCOSE        259189
HEMATOCRIT     259189
HEMOGLOBIN     259189
INR            259189
LACTATE        259189
PaCO2          259189
PLATELET       259189
POTASSIUM      259189
PT             259189
PTT            259189
SODIUM         259189
WBC            259189
dtype: int64

In [29]:
342220*20

6844400

In [30]:
5607833/6844400

0.8193315703348723

In [31]:
final.groupby('subject_id').count()['charttime']%20

subject_id
4        0
9        0
12       0
19       0
21       0
        ..
99973    0
99982    0
99985    0
99991    0
99995    0
Name: charttime, Length: 17111, dtype: int64

In [32]:
#print(maskTrain['interval'].isna().sum())
#print(activityTrain['Time'].isna().sum())

In [33]:
final.head()

,subject_id,charttime,ALBUMIN,ANION GAP,BANDS,BICARBONATE,BILIRUBIN,BUN,CHLORIDE,CREATININE,...,HEMOGLOBIN,INR,LACTATE,PaCO2,PLATELET,POTASSIUM,PT,PTT,SODIUM,WBC
0,4,2191-03-15T14:12:00Z,0.0,17.0,0.0,24.0,2.2,9.0,97.0,0.5,...,11.5,0.0,0.0,0.0,207.0,3.1,0.0,0.0,135.0,9.7
1,4,2191-03-15T15:50:00Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,12.3,31.3,0.0,0.0
2,4,2191-03-15T21:23:00Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.5,0.0,2.1,28.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,2191-03-16T05:42:00Z,2.8,15.0,0.0,21.0,1.9,10.0,108.0,0.5,...,10.6,1.1,0.0,0.0,201.0,3.3,12.8,33.2,141.0,9.7
4,4,2191-03-17T06:00:00Z,0.0,15.0,0.0,21.0,0.0,16.0,106.0,0.4,...,10.5,0.0,0.0,0.0,258.0,3.8,0.0,0.0,138.0,7.3


In [34]:
final = final.fillna(0)

In [35]:
final[final['subject_id']==4]

,subject_id,charttime,ALBUMIN,ANION GAP,BANDS,BICARBONATE,BILIRUBIN,BUN,CHLORIDE,CREATININE,...,HEMOGLOBIN,INR,LACTATE,PaCO2,PLATELET,POTASSIUM,PT,PTT,SODIUM,WBC
0,4,2191-03-15T14:12:00Z,0.0,17.0,0.0,24.0,2.2,9.0,97.0,0.5,...,11.5,0.0,0.0,0.0,207.0,3.1,0.0,0.0,135.0,9.7
1,4,2191-03-15T15:50:00Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,12.3,31.3,0.0,0.0
2,4,2191-03-15T21:23:00Z,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.5,0.0,2.1,28.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,2191-03-16T05:42:00Z,2.8,15.0,0.0,21.0,1.9,10.0,108.0,0.5,...,10.6,1.1,0.0,0.0,201.0,3.3,12.8,33.2,141.0,9.7
4,4,2191-03-17T06:00:00Z,0.0,15.0,0.0,21.0,0.0,16.0,106.0,0.4,...,10.5,0.0,0.0,0.0,258.0,3.8,0.0,0.0,138.0,7.3
5,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,4,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
ids = final['subject_id'].unique()

In [37]:
X_train, X_test = train_test_split(ids, test_size=0.20, random_state=42)

In [38]:
X_train, X_val = train_test_split(X_train, test_size=0.10, random_state=52)

In [39]:
mimicTrain=final[final['subject_id'].isin(X_train)]
mimicVal=final[final['subject_id'].isin(X_val)]
mimicTest=final[final['subject_id'].isin(X_test)]

In [40]:
#maskTrain = pd.DataFrame(np.ones((activityTrain.shape[0], activityTrain.shape[1])),columns=activityTrain.columns)
#maskTrain['Time']=activityTrain['Time']

In [41]:
maskTrain=mimicTrain[['subject_id','charttime','WBC']].copy()
maskTrain=maskTrain.fillna(0)
maskTrain[maskTrain['WBC']!=0]=1
maskTrain['charttime']=mimicTrain['charttime'].copy()
maskTrain['subject_id']=mimicTrain['subject_id'].copy()

In [42]:
maskVal=mimicVal[['subject_id','charttime','WBC']].copy()
maskVal=maskVal.fillna(0)
maskVal[maskVal['WBC']!=0]=1
maskVal['charttime']=mimicVal['charttime'].copy()
maskVal['subject_id']=mimicVal['subject_id'].copy()

In [43]:
maskTest=mimicTest[['subject_id','charttime','WBC']].copy()
maskTest=maskTest.fillna(0)
maskTest[maskTest['WBC']!=0]=1
maskTest['charttime']=mimicTest['charttime'].copy()
maskTest['subject_id']=mimicTest['subject_id'].copy()

In [44]:
maskTest.head()

,subject_id,charttime,WBC
0,4,2191-03-15T14:12:00Z,1.0
1,4,2191-03-15T15:50:00Z,0.0
2,4,2191-03-15T21:23:00Z,0.0
3,4,2191-03-16T05:42:00Z,1.0
4,4,2191-03-17T06:00:00Z,1.0


In [45]:
maskTrain.head()

,subject_id,charttime,WBC
20,9,2149-11-09T11:20:00Z,1.0
21,9,2149-11-09T17:47:00Z,0.0
22,9,2149-11-09T19:42:00Z,0.0
23,9,2149-11-10T03:00:00Z,1.0
24,9,2149-11-10T03:04:00Z,0.0


In [52]:
#decay
def decay(data):
    #print(data.head())
    data['interval']=0
    #df=data.groupby('person_id')
    j=0
    for n in range(int(data.shape[0]/20)):
        i=0
        #print(n)
        df_group=data.iloc[n*20:(n*20)+20,:]
        for index, row in df_group.iterrows():#go over mask
            #print(row['charttime'])
            if(i==0):
                row['interval']=0
                i=1
            else:
                if(row['charttime']==0):
                    row['interval']=3600+prev['interval']
                elif(prev['WBC']==1):
                    row['interval']=timedelta.total_seconds(datetime.strptime(str(row['charttime']),"%Y-%m-%dT%H:%M:%SZ")-datetime.strptime(str(prev['charttime']),"%Y-%m-%dT%H:%M:%SZ"))
                elif(prev['WBC']==0):
                    row['interval']=timedelta.total_seconds(datetime.strptime(str(row['charttime']),"%Y-%m-%dT%H:%M:%SZ")-datetime.strptime(str(prev['charttime']),"%Y-%m-%dT%H:%M:%SZ"))+prev['interval']
            prev=row
            data.iloc[j,3]=row['interval']
            j=j+1

    data['interval'] = data['interval'].apply(lambda x: abs(x)/60)
    #print(data.head())
    return data

In [53]:
def rdecay(data):
    #print(data.head())
    data['intervalReverse']=0
    j=data.shape[0]-1
    for n in range(int(data.shape[0]/20)):
        i=0
        df_group=data.iloc[n*20:(n*20)+20,:]
        df_group=df_group[::-1]
        for index, row in df_group.iterrows():#go over mask
            if(i==0):
                row['intervalReverse']=0
                i=1
            else:
                if(prev['charttime']==0):
                    row['intervalReverse']=3600+prev['intervalReverse']
                elif(prev['WBC']==1):
                    row['intervalReverse']=timedelta.total_seconds(datetime.strptime(str(row['charttime']),"%Y-%m-%dT%H:%M:%SZ")-datetime.strptime(str(prev['charttime']),"%Y-%m-%dT%H:%M:%SZ"))
                elif(prev['WBC']==0):
                    row['intervalReverse']=timedelta.total_seconds(datetime.strptime(str(row['charttime']),"%Y-%m-%dT%H:%M:%SZ")-datetime.strptime(str(prev['charttime']),"%Y-%m-%dT%H:%M:%SZ"))+prev['interval']
            prev=row
            data.iloc[j,4]=row['intervalReverse']
            j=j-1

    data['intervalReverse'] = data['intervalReverse'].apply(lambda x: abs(x)/60)
    #print(data.head())
    return data

In [54]:
maskTrain=decay(maskTrain)
print(maskTrain.head())
maskTrain=rdecay(maskTrain)

    subject_id             charttime  WBC  interval  intervalReverse
20           9  2149-11-09T11:20:00Z  1.0       0.0              0.0
21           9  2149-11-09T17:47:00Z  0.0     387.0              0.0
22           9  2149-11-09T19:42:00Z  0.0     502.0              0.0
23           9  2149-11-10T03:00:00Z  1.0     940.0              0.0
24           9  2149-11-10T03:04:00Z  0.0       4.0              0.0


In [55]:
maskVal=decay(maskVal)
print(maskVal.head())
maskVal=rdecay(maskVal)

     subject_id             charttime  WBC  interval  intervalReverse
120          25  2160-11-02T01:05:00Z  1.0       0.0              0.0
121          25  2160-11-02T03:59:00Z  0.0     174.0              0.0
122          25  2160-11-02T06:05:00Z  0.0     300.0              0.0
123          25  2160-11-02T06:56:00Z  1.0     351.0              0.0
124          25  2160-11-02T07:59:00Z  0.0      63.0              0.0


In [56]:
maskTest=decay(maskTest)
print(maskTest.head())
maskTest=rdecay(maskTest)

   subject_id             charttime  WBC  interval  intervalReverse
0           4  2191-03-15T14:12:00Z  1.0       0.0              0.0
1           4  2191-03-15T15:50:00Z  0.0      98.0              0.0
2           4  2191-03-15T21:23:00Z  0.0     431.0              0.0
3           4  2191-03-16T05:42:00Z  1.0     930.0              0.0
4           4  2191-03-17T06:00:00Z  1.0    1458.0              0.0


In [58]:
mimicTrain.to_csv('.../aaai/data/mimic/preprocess/mimicTrain.csv',index=False)
maskTrain.to_csv('.../aaai/data/mimic/preprocess/mimicTrainMask.csv',index=False)

In [59]:
mimicVal.to_csv('.../aaai/data/mimic/preprocess/mimicVal.csv',index=False)
maskVal.to_csv('.../aaai/data/mimic/preprocess/mimicValMask.csv',index=False)

In [60]:
mimicTest.to_csv('.../aaai/data/mimic/preprocess/mimicTest.csv',index=False)
maskTest.to_csv('.../aaai/data/mimic/preprocess/mimicTestMask.csv',index=False)

In [57]:
print(maskTest.head())

   subject_id             charttime  WBC  interval  intervalReverse
0           4  2191-03-15T14:12:00Z  1.0       0.0        11.800000
1           4  2191-03-15T15:50:00Z  0.0      98.0       121.733333
2           4  2191-03-15T21:23:00Z  0.0     431.0        59.683333
3           4  2191-03-16T05:42:00Z  1.0     930.0        57.650000
4           4  2191-03-17T06:00:00Z  1.0    1458.0       101.000000
